In [77]:
import graphlab
import math
import numpy as np

In [78]:
sales = graphlab.SFrame('kc_house_data.gl/')

Next write a function that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function should return a features_matrix (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It should also return an output_array which is an array of the values of the output in the data set (e.g. ‘price’). e.g. if you’re using SFrames and numpy you can complete the following function:

In [79]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 #add a constant to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the 'features' list into the SFrame 'features_sframe'
    features_sframe = data_sframe[features]
    # this will convert the features_sframe to a numpy matrix with GraphLab Create >= 1.7
    features_matrix = features_sframe.to_numpy()
    # assigne the column of data_sframe associated with the target to the variable 'output_sarray'
    output_sarray = data_sframe[output]
    #this will conver the SArray into a numpy array
    output_array = output_sarray.to_numpy()
    return (features_matrix, output_array)

In [80]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0*1.0 + 1.0*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [81]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features matrix and the weights vector. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

 If the features matrix (including a column of 1s for the constant) is stored as a 2D array (or matrix) and the regression weights are stored as a 1D array then the predicted output is just the dot product between the features matrix and the weights (with the weights on the right). Write a function ‘predict_output’ which accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and returns a 1D array ‘predictions’. e.g. in python:

In [82]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return (predictions)

In [83]:
# test the function above
test_predictions = predict_outcome(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


# Computing the Derivative

If we have a the values of a single input feature in an array ‘feature’ and the prediction ‘errors’ (predictions - output) then the derivative of the regression cost function with respect to the weight of ‘feature’ is just twice the dot product between ‘feature’ and ‘errors’. Write a function that accepts a ‘feature’ array and ‘error’ array and returns the ‘derivative’ (a single number). e.g. in python:

In [84]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(feature, errors)
    return (derivative)

In [85]:
# test the functions above
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_outcome(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of increase and therefore the negative gradient is the direction of decrease and we're trying to minimize a cost function.

The amount by which we move in the negative gradient direction is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [86]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

Now we will use our predict_output and feature_derivative to write a gradient descent function. Although we can compute the derivative for all the features simultaneously (the gradient) we will explicitly loop over the features individually for simplicity. Write a gradient descent function that does the following:

In [87]:
#Accepts a numpy feature_matrix 2D array, a 1D output array, an array of initial weights, 
# a step size and a convergence tolerance.

def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    
    #While not converged updates each feature weight by subtracting 
    #the step size times the derivative for that feature given the current weights
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = predict_outcome(feature_matrix, weights)
         # compute the errors as predictions - output
        errors = predictions - output
        
        gradient_sum_squares = 0 # intialize the gradient
        #while not converged, update each weight individually:
        for i in range(len(weights)):
            #recall that feature_matrix[:, i] is the feature column associated with weights [i]
            #compute the derivate for weight[i]
            derivative = feature_derivative(errors, feature_matrix[:, i])
            ## add the squared derivative to the gradient magnitude
            # add the squared value of the derivative to the gradient magnitude (for assessing convergence)
            gradient_sum_squares += np.square(derivative)
            
            # update the weight based on step size and derivative:
            # subtract the step size times the derivative from the current [i] weight
            weights[i] = weights[i] - (step_size * derivative)
            
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return (weights)

# Running the Gradient Descent as Simple Regression - model 1

Now split the sales data into training and test data. Like previous notebooks it’s important to use the same seed.

In [88]:
train_data, test_data = sales.random_split(.8, seed=0)

Now we will run the regression_gradient_descent function on some actual data. In particular we will use the gradient descent to estimate the model from Week 1 using just an intercept and slope. Use the following parameters:

In [89]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Use these parameters to estimate the slope and intercept for predicting prices based only on ‘sqft_living’.

e.g. using python:

In [90]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,tolerance)

In [91]:
print simple_weights

[-46999.88716555    281.91211912]


Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?

Now build a corresponding ‘test_simple_feature_matrix’ and ‘test_output’ using test_data. Using ‘test_simple_feature_matrix’ and ‘simple_weights’ compute the predicted house prices on all the test data.

In [92]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

In [93]:
test_simple_weights = regression_gradient_descent(test_simple_feature_matrix, test_output,initial_weights, step_size,tolerance)

Quiz Question: What is the predicted price for the 1st house in the Test data set for model 1 (round to nearest dollar)?

Now compute RSS on all test data for this model. Record the value and store it for later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [103]:
predicted_price = simple_weights[0] + simple_weights[1] * test_data['sqft_living']
term = test_output - predicted_price
RSS = (term **2).sum()

In [104]:
print RSS

2.75400047593e+14


In [105]:
print predicted_price[0]

356134.443171


In [107]:
predicted_price = predict_outcome(test_simple_feature_matrix, simple_weights)
print round(predicted_price[0])

356134.0


# Running a multiple regression

Now we will use the gradient descent to fit a model with more than 1 predictor variable (and an intercept). Use the following parameters:


In [108]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Run gradient descent on a model with ‘sqft_living’ and ‘sqft_living_15’ as well as an intercept with the above parameters. Save the resulting regression weights.

In [109]:
multiple_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size,tolerance)

In [110]:
print multiple_weights

[ -9.99999688e+04   2.45072603e+02   6.52795277e+01]


Use the regression weights from this second model (using sqft_living and sqft_living_15) and predict the outcome of all the house prices on the TEST data.

(Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!)

In [111]:
(test_multiple_feature_matrix, multiple_test_output) = get_numpy_data(test_data, model_features, my_output)

In [123]:
#either one works
multiple_predicted_price = multiple_weights[0] + multiple_weights[1] * test_data['sqft_living'] + multiple_weights[2]* test_data['sqft_living15']
multiple_predicted_price = predict_outcome(test_multiple_feature_matrix, multiple_weights)

Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?

In [124]:
multiple_predicted_price[0]

366651.41203655908

What is the actual price for the 1st house in the Test data set?

In [132]:
multiple_test_output[0]

310000.0

Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?

In [133]:
#model 1
predicted_price[0] - multiple_test_output[0]

46134.443170929677

In [134]:
#model 2
multiple_predicted_price[0] - multiple_test_output[0]

56651.412036559079

Now compute RSS on all test data for the second model. Record the value and store it for later.

In [128]:
term2 = multiple_test_output - multiple_predicted_price
RSS2 = (term2 * term2).sum()

In [129]:
print RSS2

2.70263446465e+14


Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data?

RSS1: 2.75395693978e+14, RSS2:5.41377498949e+14

In [130]:
print format(RSS, '.8f')

275400047593155.93750000


In [131]:
RSS2-RSS

-5136601127911.875